<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/softbutterfly.png" />
</div>

# New Relic Playground 2: Dashboards

## Imports

Python Imports

In [ ]:
import os

Third-party libraries

In [ ]:
import dotenv
import requests
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.enums import EntitySearchQueryBuilderType, DashboardEntityPermissions
from newrelic_sb_sdk.graphql.input_objects import (
    EntitySearchQueryBuilder,
    DashboardInput,
    DashboardPageInput,
    DashboardWidgetInput,
    DashboardWidgetConfigurationInput,
    DashboardMarkdownWidgetConfigurationInput
)
from newrelic_sb_sdk.graphql.objects import (
    DashboardEntity,
    DashboardEntityOutline,
    RootMutationType,
    RootQueryType,
)
from newrelic_sb_sdk.utils.query import NULL_CURSOR
from newrelic_sb_sdk.utils.response import get_response_data, print_response
from sgqlc.operation import Operation

## Client setup

To setup the client, first we need to open load the credentials from environment variables.

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

With environment varaibles loaded, we can proceed to instantiate the client

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

## Client testing

In order to use an test the client we need configure `query_type` and `mutation_type` for the `nerdgraph` schema

In [ ]:
nerdgraph.query_type = RootQueryType
nerdgraph.mutation_type = RootMutationType

For testing we will use a simple query in GraphQL to get the atttributes from our user

```gql
query {
  actor {
    user {
      email
      id
      name
    }
  }
}
```

This query will be build from the `nerdgraph` schema.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.user.__fields__("id", "email", "name")

This operation can be transformed into a GraphQL query

In [ ]:
query = operation.__to_graphql__()
print(query)

And this query is the one we send to be executed by our client.

In [ ]:
response = newrelic.execute(query)

The response data obtained is

In [ ]:
print_response(response)

We can also use the raw query directly written by hand. For this is recomendable to use the `build_query` method in order to get a clean query.

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

And execute directly, as in the previous execution, with the same result

In [ ]:
response = newrelic.execute(query)
print_response(response)

## Playground area

### Dashboards management tasks with Nerdgraph

#### Example 1: List all available dashboards

Using a raw NRQL query we can list all available dashboards with this

In [ ]:
template = """
    {
        actor {
            entitySearch(
                queryBuilder: {
                    type: DASHBOARD
                }
            ) {
                count
                results(cursor: %(cursor)s) {
                    nextCursor
                    entities {
                        ... on DashboardEntityOutline {
                            guid
                            name
                            owner {
                                email
                                userId
                            }
                            indexedAt
                            permissions
                        }
                    }
                }
            }
        }
    }
"""

params = {
    "cursor": NULL_CURSOR,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard_count = get_response_data(
    response,
    key_path="entitySearch:count",
    action="actor",
)
dashboard_count

In [ ]:
dashboards_list = get_response_data(
    response,
    key_path="entitySearch:results:entities",
    action="actor",
)
len(dashboards_list)

In [ ]:
dashboards_list[0]

In [ ]:
next_cursor = get_response_data(
    response,
    key_path="entitySearch:results:nextCursor",
    action="actor",
)
next_cursor

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

entity_search_op = operation.actor.entity_search(
    query_builder=EntitySearchQueryBuilder(type=EntitySearchQueryBuilderType.DASHBOARD)
)
entity_search_op.count()

entity_search_results_op = entity_search_op.results(cursor=None)
entity_search_results_op.next_cursor()
entity_search_results_op.entities.__as__(DashboardEntityOutline).__fields__(
    "guid",
    "name",
    "owner",
    "indexed_at",
    "permissions",
)

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
len(data.actor.entity_search.results.entities)

In [ ]:
data.actor.entity_search.results.entities[0]["guid"]

In [ ]:
data.actor.entity_search.results.next_cursor

#### Example 2: Get a single dashboard

For getting a single dashboard is required know the identification this

In [ ]:
# Getting guid the result using a raw NRQL query
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
dashboard_guid_op = data.actor.entity_search.results.entities[0]["guid"]

In [ ]:
dashboard_guid

Using a raw NRQL query we can getting a single dashboard with this

In [ ]:
template = """
    {
        actor {
            entity(guid: %(guid)s) {
                ... on DashboardEntity {
                    guid
                    name
                    owner {
                        email
                        userId
                    }
                    permissions
                    indexedAt
                }
            }
        }
    }
"""

params = {
    "guid": f'"{dashboard_guid}"',
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(
    response,
    key_path="entity",
    action="actor",
)
dashboard

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)
(
    operation.actor.entity(guid=dashboard_guid)
    .__as__(DashboardEntity)
    .__fields__
    (
        "guid",
        "name",
        "owner",
        "permissions",
        "indexed_at"
    )
)
operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
response.json()

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.entity

#### Example 3: Get a dashboard with pages

For geeting a dashboard with pages is required know the identification this

In [ ]:
# Getting guid the result using a raw NRQL query
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
dashboard_guid_op = data.actor.entity_search.results.entities[0]["guid"]

In [ ]:
dashboard_guid

Using a raw NRQL query we can get a dashboard with pages, with this

In [ ]:
template = """
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
                pages {
                    guid
                    name
                }
            }
        }
    }
}
"""

params = {
    "guid": f'"{dashboard_guid}"',
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(
    response,
    key_path="entity",
    action="actor",
)
dashboard

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation=Operation(nerdgraph.query_type)
dashboard_search_op=operation.actor.entity(guid=dashboard_guid)
dashboard_search_results_op=(
    dashboard_search_op.__as__(DashboardEntity)
    .__fields__
    (
        "guid",
        "name",
        "owner",
        "permissions",
        "indexed_at"
    ) 
)
dashboard_search_results_op=(
    dashboard_search_op.__as__(DashboardEntity).pages()
    .__fields__
    (
        "guid",
        "name",
    )
)



operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
response.json()

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.entity.pages

#### Example 4: Get a dashboard with pages and widget

For geeting a dashboard with pages and widget is required know the identification this

In [ ]:
# Getting guid the result using a raw NRQL query
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
dashboard_guid_op = data.actor.entity_search.results.entities[0]["guid"]

Using a raw NRQL query we can get a dashboard with pages and widget with this

In [ ]:
template = """
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
                pages {
                    guid
                    name
                    widgets {
                        id
                        rawConfiguration
                        title
                        visualization {
                            id
                        }
                    }
                }
            }
        }
    }
}
"""

params = {
    "guid": f'"{dashboard_guid}"',
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(
    response,
    key_path="entity",
    action="actor",
)
dashboard.keys()

In [ ]:
{key: value for key, value in dashboard.items() if key not in ("pages")}

In [ ]:
dashboard_pages = dashboard["pages"]
len(dashboard_pages)

In [ ]:
dashboard_page_0 = dashboard_pages[0]
dashboard_page_0.keys()

In [ ]:
{key: value for key, value in dashboard_page_0.items() if key not in ("widgets")}

In [ ]:
dashboard_page_0_widgets = dashboard_page_0["widgets"]
len(dashboard_page_0_widgets)

In [ ]:
dashboard_page_0_widget_0 = dashboard_page_0_widgets[0]
dashboard_page_0_widget_0

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation=Operation(nerdgraph.query_type)
dashboard_search_op=operation.actor.entity(guid=dashboard_guid)
dashboard_search_results_op=(
    dashboard_search_op.__as__(DashboardEntity)
    .__fields__
    (
        "guid",
        "name",
        "owner",
        "permissions",
        "indexed_at"
    )
)
dashboard_search_results_op=(
    dashboard_search_op.__as__(DashboardEntity).pages()
    .__fields__
    (
        "guid",
        "name",
    )
)
dashboard_search_results_op=(
    dashboard_search_op.__as__(DashboardEntity).pages().widgets()
    .__fields__
    (
        "id",
        "raw_configuration",
        "title",
        "visualization" 
        
    )
)

operation

In [ ]:
response=newrelic.execute(operation.__to_graphql__())

In [ ]:
response.json()

In [ ]:
data= operation + response.json()

In [ ]:
data.actor.entity.pages[0].widgets[0]

#### Example 5: Create a dashboard

Using a raw NRQL query we can Create a dashboard with this

In [ ]:
template = """
    mutation {
        dashboardCreate(
            accountId: %(account_id)d,
            dashboard: {
                name: "Sample api dashboard",
                pages: {
                    name: "Sample api page",
                    widgets: {
                        configuration: {
                            markdown: {
                                text: "# Sample api widget"
                            }
                        },
                        title: ""
                    }
                },
                permissions: PUBLIC_READ_WRITE
            }
        ) {
            entityResult {
                name
                owner {
                    email
                    userId
                }
                guid
            }
            errors {
                description
                type
            }
        }
    }
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
errors = get_response_data(
    response,
    key_path="errors",
    action="dashboardCreate",
)
errors

In [ ]:
created_dashboard = get_response_data(
    response,
    key_path="entityResult",
    action="dashboardCreate",
)
created_dashboard

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `mutation_type` in `nerdgraph`
operation = Operation(nerdgraph.mutation_type)
dashboard_mutate_op=(
    operation.dashboard_create(
        account_id= int(NEW_RELIC_ACCOUNT_ID),
        dashboard=DashboardInput(
            name="Sample api dashboard with Operation",
            pages=[DashboardPageInput(
                name="Sample api page with Operation",
                widgets=[DashboardWidgetInput(
                    configuration=DashboardWidgetConfigurationInput(
                        markdown=DashboardMarkdownWidgetConfigurationInput(
                            text="# Sample api widget operation"
                        )
                    ),
                    title=""
                )
                        ]
            )
                  ],
            permissions=DashboardEntityPermissions.PUBLIC_READ_WRITE
        )
    )
)

dashboard_mutate_result_op=(
    dashboard_mutate_op.entity_result()
    .__fields__
    (
        "name",
        "owner",
        "guid",
    )
)
dashboard_mutate_result_op=(
    dashboard_mutate_op.errors()
)

operation

In [ ]:
response=newrelic.execute(operation.__to_graphql__())

In [ ]:
response.json()

### Tasks

#### Task 1: Create dashboard page snapshot

Read the documentation carefully and create a snapshot of an existing dashboard

In [ ]:
# Put your code here

# Define your query template
template = """
"""

# Define your query parameters if necessary
params = {}


query = newrelic.build_query(
    template,
    params=params,
)

response = newrelic.execute(query)

In [ ]:
# only interested on the dashboard url
snapshot_url = get_response_data(
    response,
    action="dashboardCreateSnapshotUrl",
)
print(snapshot_url)

# replace PDF with PNG, and get the link to download the file
url_png = snapshot_url[:-3] + "PNG"
print(url_png)

# rename the downloaded file, and save it in the working directory
dashboard_response = requests.get(
    url_png,
    stream=True,
)

with open("dashboard_example.png", "wb") as file_handler:
    file_handler.write(dashboard_response.content)